<a href="https://colab.research.google.com/github/ElizaLo/Practice/blob/master/Encoding/Hamming%20Code/Hamming_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Hamming code

In [0]:
import random

In [0]:
# coding block length
BLOCK_LENGTH = 8

# check block length
assert not BLOCK_LENGTH % 8, 'Block length must be a multiple of 8'

# calculation of check bits / error-correcting bits, (вычисление контрольных бит)
CHECK_BITS = [i for i in range(1, BLOCK_LENGTH + 1) if not i & (i - 1)]

***Convert characters to binary format***

> *Преобразование символов в бинарный формат*





In [0]:
def chars_to_bin(chars):
    """
    Convert characters to binary format
    """
    assert not len(chars) * 8 % BLOCK_LENGTH, 'The length of the encoded data must be a multiple of the length of the coding block'
    return ''.join([bin(ord(c))[2:].zfill(8) for c in chars])

***Block output of binary data***

> *Поблочный вывод бинарных данных*


In [0]:
def block_iterator(text_bin, block_size=BLOCK_LENGTH):
    """
    Block output of binary data (Поблочный вывод бинарных данных)
    """
    for i in range(len(text_bin)):
        if not i % block_size:
            yield text_bin[i:i + block_size]

***Retrieving control (error-correcting) bit information from a binary data block***

> *Получение информации о контрольных битах из бинарного блока данных*



In [0]:
def get_check_bits_data(value_bin):
    """
    Retrieving control bit / error-correcting bits information from a binary data block
    """
    check_bits_count_map = {k: 0 for k in CHECK_BITS}
    for index, value in enumerate(value_bin, 1):
        if int(value):
            bin_char_list = list(bin(index)[2:].zfill(8))
            bin_char_list.reverse()
            for degree in [2 ** int(i) for i, value in enumerate(bin_char_list) if int(value)]:
                check_bits_count_map[degree] += 1
    check_bits_value_map = {}
    for check_bit, count in check_bits_count_map.items():
        check_bits_value_map[check_bit] = 0 if not count % 2 else 1
    return check_bits_value_map

***Add empty control (error-correcting) bits to binary block***

> *Добавить в бинарный блок "пустые" контрольные биты*



In [0]:
def set_empty_check_bits(value_bin):
    """
    Add empty control (error-correcting) bits to binary block
    """
    for bit in CHECK_BITS:
        value_bin = value_bin[:bit - 1] + '0' + value_bin[bit - 1:]
    return value_bin

***Set the control (error-correcting) bits***

>*Установить значения контрольных бит*



In [0]:
def set_check_bits(value_bin):
    """
    Set the control (error-correcting) bits
    """
    value_bin = set_empty_check_bits(value_bin)
    check_bits_data = get_check_bits_data(value_bin)
    for check_bit, bit_value in check_bits_data.items():
        value_bin = '{0}{1}{2}'.format(
            value_bin[:check_bit - 1], bit_value, value_bin[check_bit:])
    return value_bin

***Get information about control (error-correcting) bits from a binary data block***

> *Получить информацию о контрольных битах из блока бинарных данных*



In [0]:
def get_check_bits(value_bin):
    """
    Get information about control (error-correcting) bits from a binary data block
    """
    check_bits = {}
    for index, value in enumerate(value_bin, 1):
        if index in CHECK_BITS:
            check_bits[index] = int(value)
    return check_bits

***Exclude control (error-correcting) bits information from binary data block***

> *Исключить информацию о контрольных битах из блока бинарных данных*



In [0]:
def exclude_check_bits(value_bin):
    """
    Exclude control (error-correcting) bits information from binary data block
    """
    clean_value_bin = ''
    for index, char_bin in enumerate(list(value_bin), 1):
        if index not in CHECK_BITS:
            clean_value_bin += char_bin

    return clean_value_bin

***Make a mistake in the binary data blocks***


> *Допустить ошибку в блоках бинарных данных*



In [0]:
def set_errors(encoded):
    """
    Make a mistake in the binary data blocks
    """
    result = ''
    for block in block_iterator(encoded, BLOCK_LENGTH + len(CHECK_BITS)):
        num_bit = random.randint(1, len(block))
        block = '{0}{1}{2}'.format(block[:num_bit - 1], int(block[num_bit - 1]) ^ 1, block[num_bit:])
        result += (block)
    return result

***Check and fix errors in the binary data block***

> *Проверка и исправление ошибки в блоке бинарных данных*



In [0]:
def check_and_fix_error(encoded_block):
    """
    Check and fix errors in the binary data block
    """
    check_bits_encoded = get_check_bits(encoded_block)
    check_item = exclude_check_bits(encoded_block)
    check_item = set_check_bits(check_item)
    check_bits = get_check_bits(check_item)
    if check_bits_encoded != check_bits:
        invalid_bits = []
        for check_bit_encoded, value in check_bits_encoded.items():
          if check_bits[check_bit_encoded] != value:
                invalid_bits.append(check_bit_encoded)
        num_bit = sum(invalid_bits)
        encoded_block = '{0}{1}{2}'.format(
            encoded_block[:num_bit - 1],
            int(encoded_block[num_bit - 1]) ^ 1,
            encoded_block[num_bit:])
    return encoded_block

***Get a list of indices of distinct bits***

> *Получить список индексов различающихся битов*





In [0]:
def get_diff_index_list(value_bin1, value_bin2):
    """
    Get a list of indices of distinct bits
    """
    diff_index_list = []
    for index, char_bin_items in enumerate(zip(list(value_bin1), list(value_bin2)), 1):
        if char_bin_items[0] != char_bin_items[1]:
            diff_index_list.append(index)
    return diff_index_list

***Encode data***

> *Кодирование данных*



In [0]:
def encode(source):
    """
    Encode data
    """
    text_bin = chars_to_bin(source)
    result = ''
    for block_bin in block_iterator(text_bin):
        block_bin = set_check_bits(block_bin)
        result += block_bin
    return result

***Decoding data***

> *Декодирование данных*



In [0]:
def decode(encoded, fix_errors=True):
    """
    Decoding data
    """
    decoded_value = ''
    fixed_encoded_list = []
    for encoded_block in block_iterator(encoded, BLOCK_LENGTH + len(CHECK_BITS)):
        if fix_errors:
            encoded_block = check_and_fix_error(encoded_block)
        fixed_encoded_list.append(encoded_block)

    clean_block_list = []
    for encoded_block in fixed_encoded_list:
        encoded_block = exclude_check_bits(encoded_block)
        clean_block_list.append(encoded_block)

    for clean_block in clean_block_list:
        for clean_char in [clean_block[i:i + 8] for i in range(len(clean_block)) if not i % 8]:
            decoded_value += chr(int(clean_char, 2))
    return decoded_value

In [16]:
if __name__ == '__main__':
    source = input('Укажите текст для кодирования/декодирования:')
    print('Длина блока кодирования: {0}'.format(BLOCK_LENGTH))
    print('Контрольные биты: {0}'.format(CHECK_BITS))
    encoded = encode(source)
    print('Закодированные данные: {0}'.format(encoded))
    decoded = decode(encoded)
    print('Результат декодирования: {0}'.format(decoded))
    encoded_with_error = set_errors(encoded)
    print('Допускаем ошибки в закодированных данных: {0}'.format(encoded_with_error))
    diff_index_list = get_diff_index_list(encoded, encoded_with_error)
    print('Допущены ошибки в битах: {0}'.format(diff_index_list))
    decoded = decode(encoded_with_error, fix_errors=False)
    print('Результат декодирования ошибочных данных без исправления ошибок: {0}'.format(decoded))
    decoded = decode(encoded_with_error)
    print('Результат декодирования ошибочных данных с исправлением ошибок: {0}'.format(decoded))

Укажите текст для кодирования/декодирования:I saw you in Paris last summer
Длина блока кодирования: 8
Контрольные биты: [1, 2, 4, 8]
Закодированные данные: 000010001001010101000000110011100011110111010001100011110111010101000000100011101001110111001111010011100101010101000000010111001001110011011110010101000000010010100000110111010001110111110010010111001001110011100011010101000000000011001100110111010001110011100011010111110100010101000000110011100011010011100101000111011101000111011101100111000101110111110010
Результат декодирования: I saw you in Paris last summer
Допускаем ошибки в закодированных данных: 000010001011010001000000100011100011010111010001100001110111010101000100100011001001110111001101010011100100010101001000010111011001110011111110010101000001000010100000110110010001110110110010000111001001110011100001010100000000001011001100110111110001110011100111011111110100110101000000110011101011010010100101000111001101001111011101100110000101010111110010
Допущены ошибки в битах:

In [17]:
if __name__ == '__main__':
    source = input('Specify text for encoding / decoding using Humming code: ')
    print('Coding block length: {0}'.format(BLOCK_LENGTH))
    print('Control (error-correcting) bits: {0}'.format(CHECK_BITS))
    encoded = encode(source)
    print('Encoded data: {0}'.format(encoded))
    decoded = decode(encoded)
    print('Decoding result: {0}'.format(decoded))
    encoded_with_error = set_errors(encoded)
    print('Make errors in the encoded data: {0}'.format(encoded_with_error))
    diff_index_list = get_diff_index_list(encoded, encoded_with_error)
    print('Bit errors were made: {0}'.format(diff_index_list))
    decoded = decode(encoded_with_error, fix_errors=False)
    print('The result of decoding erroneous data without error correction: {0}'.format(decoded))
    decoded = decode(encoded_with_error)
    print('The result of decoding erroneous data with error correction: {0}'.format(decoded))

Specify text for encoding / decoding using Humming code: Let's leave this town tonight
Coding block length: 8
Control (error-correcting) bits: [1, 2, 4, 8]
Encoded data: 010110001100100111000101010111110100110001010111110011100011010101000000000011001100100111000101110111010001100111100110100111000101010101000000010111110100010011011000010111001001110011100011010101000000010111110100110111001111100011110111110011011110010101000000010111110100110111001111110011011110010111001001010111010111010011011000010111110100
Decoding result: Let's leave this town tonight
Make errors in the encoded data: 010100001100100111100101000111110100110001010110110011000011010101100000000011001101100111001101100111010001100011100110100111010101000101000000010101110100010011010000010111101001110011000011000101000000010011110100110111011111100011111111110011011010110101000000011111110100110111000111110010011110010111000001010111010110010011011010000111110100
Bit errors were made: [5, 19, 26, 48, 55, 67, 84, 93